In [11]:
import requests

from bs4 import BeautifulSoup, NavigableString
from html_table_extractor.extractor import Extractor
import pandas as pd
import time,os
import collections
import csv
collections.Callable = collections.abc.Callable

In [75]:
relative_path_pross_data = './processed_med_data_1_4000.csv'
relative_path_common_data = './mapping_med_data_1_4000.csv'
absolute_path_pross_data = os.path.join(os.getcwd(), relative_path_pross_data)
absolute_path_common_data = os.path.join(os.getcwd(), relative_path_common_data)

In [14]:
plant_data = pd.read_csv(absolute_path_pross_data, index_col=0, converters={'Plant_Use':eval})

In [15]:
plant_data

,Plant_Id,Scientific_name,Plant_Name_Mapped,Plant_Use
0,8,Abelmoschus esculentus,Abelmoschus esculentus,"[[None, Urination disorders], [None, Antipruri..."
1,9,Abelmoschus ficulneus,Abelmoschus ficulneus,"[[root, Wounds and injuries], [seed, Asthma], ..."
2,10,Abelmoschus manihot,Abelmoschus manihot,"[[root, Wounds and injuries], [root, Tuberculo..."
3,13,Abelmoschus moschatus,Abelmoschus moschatus,"[[leaf, Eye diseases], [None, Asthenia], [None..."
4,16,Abroma augusta,Abroma augusta,"[[root, Uterine diseases], [root, Amenorrhea],..."
...,...,...,...,...
986,3977,Dicliptera roxburghiana,Dicliptera chinensis,"[[leaf, Gonorrhea], [leaf, Whooping cough], [N..."
987,3980,Dicoma tomentosa,Dicoma tomentosa,"[[root, Dental caries], [whole plant, Wounds a..."
988,3982,Dicranostigma lactucoides,Dicranostigma lactucoides,"[[root, Pregnancy complications]]"
989,3983,Dictamnus albus,Dictamnus albus,"[[root, Hysteria], [root, Nervous system disea..."


In [18]:
plant_data['Plant_Id'].nunique()

990

In [24]:
plant_data[plant_data['Plant_Id'].duplicated()]

,Plant_Id,Scientific_name,Plant_Name_Mapped,Plant_Use
834,3333,Croton laccifer,Macaranga tanarius,"[[whole plant excluding root, Antiviral agents..."


In [23]:
plant_data[plant_data['Plant_Id'].duplicated(keep='last')]

,Plant_Id,Scientific_name,Plant_Name_Mapped,Plant_Use
833,3333,Croton laccifer,Croton laccifer,"[[root, Antifungal agents], [root, Insecticides]]"


In [48]:
def extract_content_from_h5(soup_test):
    # Find the first h2 tag with class "title"
    h5_tag = soup_test.find('h5')
    
    # Extract the content inside the first <i> tag within the h2 tag
    if h5_tag:
        i_tags = h5_tag.find_all('strong')
        if i_tags:
            first_i_tag_content = i_tags[0].text.strip()
            return first_i_tag_content
    
    return None

In [49]:
def extract_table_content(soup_test):
    
    # Find the first table tag in the parsed HTML
    data = soup_test.find('div', class_='row')
    
    # Initialize an empty list to store the extracted content
    content_list = []
    i = 1
    # If a table tag is found, extract the content inside it
    if data:
        for item in data.stripped_strings:
            if item == 'More Information:':
                    break;
            content_list.append(item)
    
    result = []
    if len(content_list)>0:
        content_list = content_list[1:]

        # Initialize variables
        current_element = ""

        # Process the data
        for item in content_list:
            if ":" in item:
                if current_element:
                    result.append([current_element])
                current_element = item
            else:
                current_element += item

        # Add the last element if any
        if current_element:
            result.append([current_element])

    
    return result

In [63]:
# Define the base URL pattern
base_url = "https://cb.imsc.res.in/imppat/therapeutics/{}"

# Define the User-Agent header
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Initialize an empty list to store the extracted data
all_data = []


# # Loop through IDs from 1 to 13122
for index, row in plant_data.iterrows():
    
    # Construct the URL with the current herb ID
    url = base_url.format(row['Plant_Name_Mapped'])
    
    # Send an HTTP GET request to the website with the defined headers
    response = requests.get(url, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML code using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        one_plant=[]
        pid = 'Plant_id: ' + str(row['Plant_Id'])
        plant_name = 'Mapped_name:' + extract_content_from_h5(soup)
        one_plant.append([pid])
        one_plant.append([plant_name])
        one_plant.extend(extract_table_content(soup))        
        # Append the dictionary to the list
        all_data.append(one_plant)
    else:
        print(f"Failed to retrieve data for ID {herb_id}. Status code: {response.status_code}")

    # Adding a delay between requests to avoid rate limiting
    time.sleep(1)  # Sleep for 1 second between requests


In [64]:
all_data

[[['Plant_id: 8'],
  ['Mapped_name:Abelmoschus esculentus'],
  ['Kingdom:Plantae'],
  ['Family:Malvaceae'],
  ['Group:Angiosperms'],
  ['Common name:Ladies Finger'],
  ['Synonymous names:Hibiscus esculentus, Abelmoschus esculentus'],
  ['System of medicine:Ayurveda, Siddha, Unani']],
 [['Plant_id: 9'],
  ['Mapped_name:Abelmoschus ficulneus'],
  ['Kingdom:Plantae'],
  ['Family:Malvaceae'],
  ['Group:Angiosperms'],
  ['Common name:White Wild Musk Mallow'],
  ['Synonymous names:Hibiscus ficulneus, Abelmoschus ficulneus'],
  ['System of medicine:Ayurveda, Siddha'],
  ['IUCN Red List category:Least Concern']],
 [['Plant_id: 10'],
  ['Mapped_name:Abelmoschus manihot'],
  ['Kingdom:Plantae'],
  ['Family:Malvaceae'],
  ['Group:Angiosperms'],
  ['Common name:Sweet Hibiscus'],
  ['Synonymous names:Hibiscus manihot, Abelmoschus manihot']],
 [['Plant_id: 13'],
  ['Mapped_name:Abelmoschus moschatus'],
  ['Kingdom:Plantae'],
  ['Family:Malvaceae'],
  ['Group:Angiosperms'],
  ['Common name:Pink Swamp

In [65]:
len(all_data)

991

In [66]:
def extract_data_to_csv(scraped_data):
    # Define the header row for the CSV file
    header = ['Plant_Id', 'Mapped_Name', 'Kingdom', 'Family', 'Group', 'Common_Name', 'Synonymous_names', 'System_of_medicine']
    
    # Initialize an empty list to store data rows
    data_rows = []
        
    for item in scraped_data:
        # Initialize variables to store extracted information
        pid = ''
        mapped_name = ''
        kingdom = ''
        family = ''
        group = ''
        common_name = ''
        syn_name = ''
        sys_med = ''
        
            # Loop through each item in the scraped data
        for sub_item in range(len(item)):
                
            # Extract information based on the content of the sub-item
            if item[sub_item][0].startswith('Plant_id:'):
                pid = item[sub_item][0].split(':')[1].strip()
            elif item[sub_item][0].startswith('Mapped_name:'):
                mapped_name = item[sub_item][0].split(':')[1].strip()
            elif item[sub_item][0].startswith('Kingdom:'):
                kingdom = item[sub_item][0].split(':')[1].strip()
            elif item[sub_item][0].startswith('Family:'):
                family = item[sub_item][0].split(':')[1].strip()
            elif item[sub_item][0].startswith('Group:'):
                group = item[sub_item][0].split(':')[1].strip()
            elif item[sub_item][0].startswith('Common name:'):
                common_name = item[sub_item][0].split(':')[1].strip()
            elif item[sub_item][0].startswith('Synonymous names:'):
                syn_name = item[sub_item][0].split(':')[1].strip()
            elif item[sub_item][0].startswith('System of medicine:'):
                sys_med = item[sub_item][0].split(':')[1].strip()
                    
         # Append the extracted information as a dictionary to the data rows list
        data_rows.append({
            'Plant_Id': pid,
            'Mapped_Name': mapped_name,
            'Kingdom': kingdom,
            'Family': family,
            'Group': group,
            'Common_Name': common_name,
            'Synonymous_names': syn_name,
            'System_of_medicine': sys_med
        })
        
    # Create a Pandas DataFrame using the data rows list and header
    df = pd.DataFrame(data_rows, columns=header)
    
    return df

In [67]:
# Call the function to extract data to CSV
data_df = extract_data_to_csv(all_data)


In [68]:
data_df

,Plant_Id,Mapped_Name,Kingdom,Family,Group,Common_Name,Synonymous_names,System_of_medicine
0,8,Abelmoschus esculentus,Plantae,Malvaceae,Angiosperms,Ladies Finger,"Hibiscus esculentus, Abelmoschus esculentus","Ayurveda, Siddha, Unani"
1,9,Abelmoschus ficulneus,Plantae,Malvaceae,Angiosperms,White Wild Musk Mallow,"Hibiscus ficulneus, Abelmoschus ficulneus","Ayurveda, Siddha"
2,10,Abelmoschus manihot,Plantae,Malvaceae,Angiosperms,Sweet Hibiscus,"Hibiscus manihot, Abelmoschus manihot",
3,13,Abelmoschus moschatus,Plantae,Malvaceae,Angiosperms,Pink Swamp Mallow,"Hibiscus abelmoschus, Abelmoschus moschatus, A...","Ayurveda, Siddha, Unani"
4,16,Abroma augusta,Plantae,Malvaceae,Angiosperms,Devil's Cotton,Abroma augusta,"Ayurveda, Siddha, Unani"
...,...,...,...,...,...,...,...,...
986,3977,Dicliptera chinensis,Plantae,Acanthaceae,Angiosperms,Chinese Foldwing,"Dicliptera roxburghiana, Dicliptera chinensis",
987,3980,Dicoma tomentosa,Plantae,Compositae,Angiosperms,Woolly Dicoma,Dicoma tomentosa,
988,3982,Dicranostigma lactucoides,Plantae,Papaveraceae,Angiosperms,,Dicranostigma lactucoides,
989,3983,Dictamnus albus,Plantae,Rutaceae,Angiosperms,,"Dictamnus hispanicus, Dictamnus albus",Homeopathy


In [69]:
data_df.columns

Index(['Plant_Id', 'Mapped_Name', 'Kingdom', 'Family', 'Group', 'Common_Name',
       'Synonymous_names', 'System_of_medicine'],
      dtype='object')

In [71]:
data_merge = data_df.drop(columns=['Plant_Id', 'Mapped_Name'])
data_merge

,Kingdom,Family,Group,Common_Name,Synonymous_names,System_of_medicine
0,Plantae,Malvaceae,Angiosperms,Ladies Finger,"Hibiscus esculentus, Abelmoschus esculentus","Ayurveda, Siddha, Unani"
1,Plantae,Malvaceae,Angiosperms,White Wild Musk Mallow,"Hibiscus ficulneus, Abelmoschus ficulneus","Ayurveda, Siddha"
2,Plantae,Malvaceae,Angiosperms,Sweet Hibiscus,"Hibiscus manihot, Abelmoschus manihot",
3,Plantae,Malvaceae,Angiosperms,Pink Swamp Mallow,"Hibiscus abelmoschus, Abelmoschus moschatus, A...","Ayurveda, Siddha, Unani"
4,Plantae,Malvaceae,Angiosperms,Devil's Cotton,Abroma augusta,"Ayurveda, Siddha, Unani"
...,...,...,...,...,...,...
986,Plantae,Acanthaceae,Angiosperms,Chinese Foldwing,"Dicliptera roxburghiana, Dicliptera chinensis",
987,Plantae,Compositae,Angiosperms,Woolly Dicoma,Dicoma tomentosa,
988,Plantae,Papaveraceae,Angiosperms,,Dicranostigma lactucoides,
989,Plantae,Rutaceae,Angiosperms,,"Dictamnus hispanicus, Dictamnus albus",Homeopathy


In [73]:
# Join df2 with df1 based on index
merged_df = plant_data.join(data_merge, how='left')

merged_df

,Plant_Id,Scientific_name,Plant_Name_Mapped,Plant_Use,Kingdom,Family,Group,Common_Name,Synonymous_names,System_of_medicine
0,8,Abelmoschus esculentus,Abelmoschus esculentus,"[[None, Urination disorders], [None, Antipruri...",Plantae,Malvaceae,Angiosperms,Ladies Finger,"Hibiscus esculentus, Abelmoschus esculentus","Ayurveda, Siddha, Unani"
1,9,Abelmoschus ficulneus,Abelmoschus ficulneus,"[[root, Wounds and injuries], [seed, Asthma], ...",Plantae,Malvaceae,Angiosperms,White Wild Musk Mallow,"Hibiscus ficulneus, Abelmoschus ficulneus","Ayurveda, Siddha"
2,10,Abelmoschus manihot,Abelmoschus manihot,"[[root, Wounds and injuries], [root, Tuberculo...",Plantae,Malvaceae,Angiosperms,Sweet Hibiscus,"Hibiscus manihot, Abelmoschus manihot",
3,13,Abelmoschus moschatus,Abelmoschus moschatus,"[[leaf, Eye diseases], [None, Asthenia], [None...",Plantae,Malvaceae,Angiosperms,Pink Swamp Mallow,"Hibiscus abelmoschus, Abelmoschus moschatus, A...","Ayurveda, Siddha, Unani"
4,16,Abroma augusta,Abroma augusta,"[[root, Uterine diseases], [root, Amenorrhea],...",Plantae,Malvaceae,Angiosperms,Devil's Cotton,Abroma augusta,"Ayurveda, Siddha, Unani"
...,...,...,...,...,...,...,...,...,...,...
986,3977,Dicliptera roxburghiana,Dicliptera chinensis,"[[leaf, Gonorrhea], [leaf, Whooping cough], [N...",Plantae,Acanthaceae,Angiosperms,Chinese Foldwing,"Dicliptera roxburghiana, Dicliptera chinensis",
987,3980,Dicoma tomentosa,Dicoma tomentosa,"[[root, Dental caries], [whole plant, Wounds a...",Plantae,Compositae,Angiosperms,Woolly Dicoma,Dicoma tomentosa,
988,3982,Dicranostigma lactucoides,Dicranostigma lactucoides,"[[root, Pregnancy complications]]",Plantae,Papaveraceae,Angiosperms,,Dicranostigma lactucoides,
989,3983,Dictamnus albus,Dictamnus albus,"[[root, Hysteria], [root, Nervous system disea...",Plantae,Rutaceae,Angiosperms,,"Dictamnus hispanicus, Dictamnus albus",Homeopathy


In [76]:
merged_df.to_csv(absolute_path_common_data)